Note: this notebook is set up to run with the env.yml containing the name 'polaris_datasets'

# Dataset creation with [Polaris](https://github.com/polaris-hub/polaris) 

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pathlib

import pandas as pd
import datamol as dm

# polaris dataset
from polaris.dataset import Dataset, ColumnAnnotation

from polaris.utils.types import HubOwner


root = pathlib.Path("__file__").absolute().parents[2]
os.chdir(root)
sys.path.insert(0, str(root))
# from utils.docs_utils import load_readme

In [2]:
# Get the owner and organization
org = "Graphium"
data_name = "pm6_subset"
dirname = dm.fs.join(root, f"org-{org}", data_name)
gcp_root = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}"

owner = HubOwner(slug=org.lower(), type="organization")
owner

HubOwner(slug='graphium', external_id=None, type='organization')

In [3]:
BENCHMARK_DIR = f"{gcp_root}/benchmarks"
DATASET_DIR = f"{gcp_root}/datasets"
FIGURE_DIR = f"{gcp_root}/figures"

## Load existing data

In [4]:
PATH = f"{gcp_root}/data/raw/pm6_86M_0.parquet"
PATH = "/mnt/ps/home/CORP/lu.zhu/project/polaris-recipes/org-Graphium/pm6_83m/pm6_86M_0.parquet"
table = pd.read_parquet(PATH)
table.columns

Index(['graph_ordered_smiles', 'graph_cation_plane_best_fit',
       'graph_S0_principal_length_b', 'graph_S0_principal_length_a',
       'graph_cation_energy_total', 'graph_anion_plane_best_fit',
       'graph_T0_plane_best_fit', 'graph_anion_beta_homo',
       'graph_cation_beta_homo', 'graph_T0_alpha_homo', 'graph_S0_beta_gap',
       'graph_T0_beta_homo', 'graph_cation_principal_length_b',
       'graph_T0_spherocity', 'graph_S0_plane_best_fit',
       'graph_T0_principal_length_b', 'graph_anion_principal_length_c',
       'graph_anion_alpha_gap', 'graph_T0_principal_length_c',
       'graph_cation_principal_length_a', 'graph_S0_alpha_homo',
       'graph_cation_beta_gap', 'graph_S0_energy_total', 'graph_S0_spherocity',
       'graph_anion_principal_length_a', 'graph_T0_principal_length_a',
       'graph_anion_alpha_homo', 'graph_cation_alpha_homo',
       'graph_T0_energy_total', 'graph_S0_principal_length_c',
       'graph_anion_spherocity', 'graph_anion_principal_length_b',
    

In [10]:
table.shape

(8506975, 64)

### Below we specify the meta information of data columns

In [5]:
# Additional meta-data on the column level
annotations = {
    "graph_ordered_smiles": ColumnAnnotation(
        description="Molecule SMILES string", modality="molecule"
    ),
    **{
        col: ColumnAnnotation(description=f"Graph feature {col}")
        for col in table.columns
        if col.startswith("graph")
    },
}

### Define `Dataset` object

In [6]:
version = "v1"
dataset_name = f"{data_name}-{version}"

In [7]:
dataset = Dataset(
    # The table is the core data-structure required to construct a dataset
    table=table[annotations.keys()],
    # Additional meta-data on the dataset level.
    name=dataset_name,
    description="Subset of quantum chemistry dataset which uses PM6 semi-empirical computation of the quantum properties.",
    source="https://arxiv.org/abs/1904.06046",
    annotations=annotations,
    tags=["Graph", "Quantum chemistry", "LargeMix"],
    owner=owner,
    license="CC-BY-4.0",
    user_attributes={"year": "2019"},
    # readme=load_readme("org-Graphium/pm6_83m/readme.md"),
)

In [1]:
# save the dataset to GCP
SAVE_DIR = f"{DATASET_DIR}/{dataset_name}"
dataset_path = dataset.to_json(SAVE_DIR)
dataset_path

In [9]:
from polaris.hub.client import PolarisHubClient

client = PolarisHubClient()
client.login()

client.upload_dataset(dataset=dataset, access="public", owner=owner, timeout=5000)

2024-07-20 20:58:48.592 | SUCCESS  | polaris.hub.client:login:224 - You are successfully logged in to the Polaris Hub.
2024-07-20 21:00:40.651 | SUCCESS  | polaris.hub.client:upload_dataset:569 - Your dataset has been successfully uploaded to the Hub. View it here: https://polarishub.io/datasets/graphium/pm6_subset-v1


{'id': 'gt7Tb8ex4HWdNbYSv6Lfm',
 'createdAt': '2024-07-21T02:59:25.795Z',
 'deletedAt': None,
 'name': 'pm6-subset-v1',
 'slug': 'pm6-subset-v1',
 'description': 'Subset of quantum chemistry dataset which uses PM6 semi-empirical computation of the quantum properties.',
 'tags': ['Graph', 'Quantum chemistry', 'LargeMix'],
 'userAttributes': {'year': '2019'},
 'access': 'public',
 'isCertified': False,
 'polarisVersion': 'dev',
 'readme': '',
 'ownerId': 'zMTB7lQiiukqEmLQF7EjT',
 'creatorId': 'NKnaHGybLqwSHcaMEHqfF',
 'state': 'upload_pending',
 'source': 'https://arxiv.org/abs/1904.06046',
 'curationReference': None,
 'nRows': 8506975,
 'nColumns': 63,
 'license': 'CC-BY-4.0',
 'md5Sum': '75c117e0e2278da52f989a1c71ebf896',
 'annotations': {'graph_mw': {'dtype': 'float64',
   'modality': 'UNKNOWN',
   'isPointer': False,
   'description': 'Graph feature graph_mw',
   'userAttributes': {}},
  'graph_qed': {'dtype': 'float64',
   'modality': 'UNKNOWN',
   'isPointer': False,
   'descriptio